In [1]:
# force the notebook to auto reload external python modules | useful for development
%load_ext autoreload
%autoreload 2

In [2]:
# imports
import sys

sys.path.append("..\\")
from src.data import MyDataset, AudioTripletDataset, LandmarkTripletDataset
from src.features import TripletGenerator
# from src.models import main

import torch
from torch.utils.data import random_split
from torch.utils.data import DataLoader

random_seed = 42

In [3]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [4]:
# paths
raw_data_path = "..\\data\\raw\\"
processed_data_path = "..\\data\\processed\\"

---

In [5]:
my_dataset = MyDataset(location=raw_data_path)

In [5]:
# create & pickle dataset | RUN THIS CELL ONLY ONCE TO CREATE THE DATASET
my_dataset.create_dataset()
my_dataset.save_dataset(processed_data_path)

c:\Users\hgnis\anaconda3\Lib\site-packages\torchaudio\functional\functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [5]:
# load dataset | RUN THIS CELL TO LOAD THE SAVED DATASET
my_dataset.load_dataset(processed_data_path)
sample_size = len(my_dataset)
print(f"Loaded dataset with {sample_size} samples.")

Loaded dataset with 323 samples.


In [6]:
# train-valid-test split
# to maintain independence between train and test sets, split the dataset before creating the triplets

train_size = int(0.8 * sample_size)
val_size = int(0.1 * sample_size)
test_size = sample_size - train_size - val_size

train_set, val_set, test_set = random_split(my_dataset, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(random_seed))

train_size, valid_size, test_size = len(train_set), len(val_set), len(test_set)
print(f"Train dataset size: {train_size} \nValid dataset size: {valid_size} \nTest dataset size: {test_size}")

Train dataset size: 258 
Valid dataset size: 32 
Test dataset size: 33


In [7]:
# CREATE & PICKLE triplets of indicies | RUN THIS CELL ONLY ONCE TO CREATE THE TRIPLETS
# ensure diversity of triplets by setting num_triplets to a large number
# essentially 10 to 20 triplets per sample

# TRAIN
train_triplets = TripletGenerator(train_set, num_triplets=train_size*10, prefix="train")
train_triplets.save_triplets(processed_data_path)

# VALID
valid_triplets = TripletGenerator(val_set, num_triplets=valid_size*10, prefix="valid")
valid_triplets.save_triplets(processed_data_path)

# TEST
test_triplets = TripletGenerator(test_set, num_triplets=test_size*10, prefix="test")
test_triplets.save_triplets(processed_data_path)

print(f"Created and saved: \
      \n {len(train_triplets.triplets)} train triplets \
      \n {len(valid_triplets.triplets)} valid triplets \
      \n {len(test_triplets.triplets)} test triplets"
      )

Created and saved:       
 2580 train triplets       
 320 valid triplets       
 330 test triplets


---

In [6]:
# LOAD triplets | RUN THIS CELL TO LOAD THE SAVED TRIPLETS

# TRAIN
train_triplets = TripletGenerator(load=True, root_path=processed_data_path, prefix="train")
train_audio_triplet_dataset = AudioTripletDataset(my_dataset.data, train_triplets.triplets)
train_landmark_triplet_dataset = LandmarkTripletDataset(my_dataset.data, train_triplets.triplets)

# VALID
valid_triplets = TripletGenerator(load=True, root_path=processed_data_path, prefix="valid")
valid_audio_triplet_dataset = AudioTripletDataset(my_dataset.data, valid_triplets.triplets)
valid_landmark_triplet_dataset = LandmarkTripletDataset(my_dataset.data, valid_triplets.triplets)

# TEST
test_triplets = TripletGenerator(load=True, root_path=processed_data_path, prefix="test")
test_audio_triplet_dataset = AudioTripletDataset(my_dataset.data, test_triplets.triplets)
test_landmark_triplet_dataset = LandmarkTripletDataset(my_dataset.data, test_triplets.triplets)

print(f"Loaded: \
      \n {len(train_triplets.triplets)} train triplets \
      \n {len(valid_triplets.triplets)} valid triplets \
      \n {len(test_triplets.triplets)} test triplets"
      )

Loaded:       
 2580 train triplets       
 320 valid triplets       
 330 test triplets


In [7]:
# imports
from src.config import SystemConfig, TrainingConfig
from src.logging import setup_log_directory
from src.models import audio_model, landmark_model, combined_model, main
from src.visualization import plot_loss_accuracy

from torch.utils.tensorboard import SummaryWriter

### Audio model

In [9]:
# create dataloaders
train_loader = DataLoader(train_audio_triplet_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_audio_triplet_dataset, batch_size=32, shuffle=False)

In [11]:
# train model
model = audio_model((250, 400))
print(model)

training_config = TrainingConfig()

# Model checkpoint log dir setup.
training_config, current_version_name = setup_log_directory(training_config)

# Tensorboard log dir setup.
summary_writer = SummaryWriter(training_config.log_dir)

SiameseModel(
  (siamese_network): LSTM(
    (lstm): LSTM(400, 64, batch_first=True)
    (attention): Linear(in_features=64, out_features=1, bias=True)
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (dense1): Linear(in_features=64, out_features=128, bias=True)
    (dense2): Linear(in_features=128, out_features=64, bias=True)
    (dense3): Linear(in_features=64, out_features=32, bias=True)
  )
)
Logging at: ..\output\Logs_Checkpoints\Model_logs\version_7
Model Checkpoint at: ..\output\Logs_Checkpoints\Model_checkpoints\version_7


In [12]:
# Train and Validate
train_loss, train_acc, val_loss, val_acc = main(
    model,
    (train_loader, valid_loader),
    summary_writer=summary_writer,
    scheduler=None,
    system_config=SystemConfig(),
    training_config=training_config,
    data_augmentation=False,
)

Train:	Epoch: 1/10 Train Loss: 1.0000, Train Acc: 0.5000: 100%|██████████| 81/81 [00:25<00:00,  3.18it/s]
Valid:	Epoch: 1/10 Valid Loss: 0.9992, Valid Acc: 0.5000: 100%|██████████| 10/10 [00:01<00:00,  9.59it/s]



Model Improved... Saving Model ... Done.




Train:	Epoch: 2/10 Train Loss: 0.9980, Train Acc: 0.5000: 100%|██████████| 81/81 [00:17<00:00,  4.60it/s]
Valid:	Epoch: 2/10 Valid Loss: 0.9991, Valid Acc: 0.5000: 100%|██████████| 10/10 [00:00<00:00, 10.04it/s]



Model Improved... Saving Model ... Done.




Train:	Epoch: 3/10 Train Loss: 0.9790, Train Acc: 0.5000: 100%|██████████| 81/81 [00:22<00:00,  3.63it/s]
Valid:	Epoch: 3/10 Valid Loss: 0.9947, Valid Acc: 0.5000: 100%|██████████| 10/10 [00:01<00:00,  8.53it/s]


...

Train:	Epoch: 7/10 Train Loss: 0.0000, Train Acc: 0.9614: 100%|██████████| 81/81 [00:23<00:00,  3.42it/s]
Valid:	Epoch: 7/10 Valid Loss: 1.0844, Valid Acc: 0.5469: 100%|██████████| 10/10 [00:01<00:00,  7.19it/s]


Train:	Epoch: 8/10 Train Loss: 0.0000, Train Acc: 0.9614: 100%|██████████| 81/81 [00:24<00:00,  3.28it/s]
Valid:	Epoch: 8/10 Valid Loss: 1.0844, Valid Acc: 0.5469: 100%|██████████| 10/10 [00:01<00:00,  6.57it/s]


Train:	Epoch: 9/10 Train Loss: 0.0000, Train Acc: 0.9614: 100%|██████████| 81/81 [00:25<00:00,  3.21it/s]
Valid:	Epoch: 9/10 Valid Loss: 1.0844, Valid Acc: 0.5469: 100%|██████████| 10/10 [00:01<00:00,  7.12it/s]


Train:	Epoch: 10/10 Train Loss: 0.0000, Train Acc: 0.9614: 100%|██████████| 81/81 [00:23<00:00,  3.42it/s]
Valid:	Epoch: 10/10 Valid Loss: 1.0844, Valid Acc: 0.5469: 100%|██████████| 10/10 [00:01<00:00,  7.21it/s]


Total time: 247.28s, Best Loss: 0.995


In [13]:
plot_loss_accuracy(
    train_loss=[train_loss],
    val_loss=[val_loss],
    train_acc=[train_acc],
    val_acc=[val_acc],
    colors=["blue"],
    loss_legend_loc="upper center",
    acc_legend_loc="upper left",
)

: 

### Landmark Model

In [8]:
# create dataloaders
train_loader = DataLoader(train_landmark_triplet_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_landmark_triplet_dataset, batch_size=32, shuffle=False)

In [14]:
# train model
model = landmark_model()
print(model)

training_config = TrainingConfig()

# Model checkpoint log dir setup.
training_config, current_version_name = setup_log_directory(training_config)

# Tensorboard log dir setup.
summary_writer = SummaryWriter(training_config.log_dir)

SiameseModel(
  (siamese_network): STGCN(
    (temporal_conv1): Conv2d(3, 64, kernel_size=(3, 1), stride=(1, 1))
    (graph_conv1): GraphConvLayer(
      (fc): Linear(in_features=64, out_features=32, bias=True)
    )
    (temporal_conv2): Conv2d(32, 64, kernel_size=(3, 1), stride=(1, 1))
  )
)
Logging at: ..\output\Logs_Checkpoints\Model_logs\version_11
Model Checkpoint at: ..\output\Logs_Checkpoints\Model_checkpoints\version_11


In [ ]:
# Train and Validate
train_loss, train_acc, val_loss, val_acc = main(
    model,
    (train_loader, valid_loader),
    summary_writer=summary_writer,
    scheduler=None,
    system_config=SystemConfig(),
    training_config=training_config,
    data_augmentation=False,
)

In [ ]:
plot_loss_accuracy(
    train_loss=[train_loss],
    val_loss=[val_loss],
    train_acc=[train_acc],
    val_acc=[val_acc],
    colors=["blue"],
    loss_legend_loc="upper center",
    acc_legend_loc="upper left",
)

### Combined model

In [ ]:
# dataloaders

In [13]:
# train model
model = combined_model((250, 400))
print(model)

training_config = TrainingConfig()

# Model checkpoint log dir setup.
training_config, current_version_name = setup_log_directory(training_config)

# Tensorboard log dir setup.
summary_writer = SummaryWriter(training_config.log_dir)

CombinedSiameseNetwork(
  (audio_network): LSTM(
    (lstm): LSTM(400, 64, batch_first=True)
    (attention): Linear(in_features=64, out_features=1, bias=True)
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (dense1): Linear(in_features=64, out_features=128, bias=True)
    (dense2): Linear(in_features=128, out_features=64, bias=True)
    (dense3): Linear(in_features=64, out_features=32, bias=True)
  )
  (landmarks_network): STGCN(
    (temporal_conv1): Conv2d(3, 64, kernel_size=(3, 1), stride=(1, 1))
    (graph_conv1): GraphConvLayer(
      (fc): Linear(in_features=64, out_features=32, bias=True)
    )
    (temporal_conv2): Conv2d(32, 64, kernel_size=(3, 1), stride=(1, 1))
  )
  (fc): Linear(in_features=64, out_features=32, bias=True)
)
Logging at: ..\output\Logs_Checkpoints\Model_logs\version_10
Model Checkpoint at: ..\output\Logs_Checkpoints\Model_checkpoints\version_10


In [ ]:
# Train and Validate
train_loss, train_acc, val_loss, val_acc = main(
    model,
    (train_loader, valid_loader),
    summary_writer=summary_writer,
    scheduler=None,
    system_config=SystemConfig(),
    training_config=training_config,
    data_augmentation=False,
)

In [ ]:
plot_loss_accuracy(
    train_loss=[train_loss],
    val_loss=[val_loss],
    train_acc=[train_acc],
    val_acc=[val_acc],
    colors=["blue"],
    loss_legend_loc="upper center",
    acc_legend_loc="upper left",
)